In [ ]:
from atmospheric_explorer.cams_interfaces import InversionOptimisedGreenhouseGas, EAC4Instance
from atmospheric_explorer.shapefile import ShapefilesDownloader
from atmospheric_explorer.utils import get_local_folder
from atmospheric_explorer.units_conversion import convert_units_array
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import os
from glob import glob
import xarray as xr
from datetime import datetime
import geopandas as gpd
from shapely.geometry import mapping
import rioxarray
import plotly.graph_objects as go
import shutil
import numpy as np
import statsmodels.stats.api as sms
import pandas as pd
import cdsapi
from atmospheric_explorer.plotting_apis import eac4_hovmoeller_levels_plot
from atmospheric_explorer.data_transformations import clip_and_concat_countries, shifting_long

In [ ]:
# Remove previous data
shutil.rmtree(os.path.join(get_local_folder(), 'data'))

In [ ]:
# Remove previous data
shutil.rmtree(os.path.join(get_local_folder(), 'shapefiles'))

In [ ]:
#function to move from 0+360 to -180+180 long
def ds_swaplon(ds):
    return ds.assign_coords(longitude=(((ds.longitude + 180) % 360) - 180)).sortby('longitude')

In [ ]:
manager = InversionOptimisedGreenhouseGas(
    data_variables='carbon_dioxide',
    file_format='zip',
    quantity='surface_flux',
    input_observations='surface',
    time_aggregation='monthly_mean',
    year=[
        '1985', '1986', '1987',
        '1988', '1989', '1990',
        '1991', '1992', '1993',
        '1994', '1995', '1996',
        '1997', '1998', '1999',
        '2000', '2001', '2002',
        '2003', '2004', '2005',
        '2006', '2007', '2008',
        '2009', '2010', '2011',
        '2012', '2013', '2014',
        '2015', '2016', '2017',
        '2018', '2019', '2020'
    ],
    month=[
        '01', '02', '03',
        '04', '05', '06',
        '07', '08', '09',
        '10', '11', '12'
    ]
)
manager.download()

In [ ]:
#sorted: ordina in ordine alfabetico  - globe: dentro è una lista (casting=viene 'converitito' in una lista)
files = sorted(glob(manager.file_full_path))

In [ ]:
#cos'è un genereatore:
gen=(i for i in range(10))
#lista invece:
lista=[i for i in range(10)]

#come si scrive la lista [] o il generatore () si chiama list comprehension: permette di definire l'oggetto semplicemente con le parentesi

In [ ]:
#il generatore se lo runni dura solo il tempo del print (se cambi gen con list non è cosi invece) - se lo runni di nuovo non contiene valori
for v in gen:
    print(v)

In [ ]:
# Create dataframe with first file
mm = datetime.strptime(files[0].split('_')[-1].split('.')[0], '%Y%m')
df = xr.open_dataset(files[0])[['flux_foss']]
df = df.expand_dims({'time': [mm]})
# Merge remaining files
# ! This loop replaces xr.open_mfdataset(manager.file_full_path) that does not work (because time coordinate is not included in dataframe)
for file in files[1:]:
    mm = datetime.strptime(file.split('_')[-1].split('.')[0], '%Y%m')
    temp = xr.open_dataset(file)[['flux_foss']]
    temp = temp.expand_dims({'time': [mm]})
    df = xr.combine_by_coords([df, temp])

In [ ]:
df = df.rio.write_crs('EPSG:4326')

In [ ]:
sh_down = ShapefilesDownloader(
    resolution='10m',
    instance='countries'
)
sh_down.download_shapefile()

In [ ]:
sh = gpd.read_file(sh_down.shapefile_full_path, crs='EPSG:4326')

In [ ]:
#all_touched=True questo parametro include tutti i pixel toccati dal poligono definito, se False include solo i pixel il cui centro è incluso nel poligono
#approvato all_touched=True
df_clipped = df.rio.clip(sh[sh['ADMIN'] == 'Italy'].geometry.apply(mapping), sh.crs, drop=True, all_touched=True)[['flux_foss']]

In [ ]:
df_clipped['flux_foss'][0].plot()

In [ ]:
# Drop all values that are null over all coords, compute the mean of the remaining values over long and lat
df_clipped = df_clipped.where(~df_clipped['flux_foss'].isnull(), drop=True).sortby('time').mean(dim=['longitude', 'latitude'])

In [ ]:
da_converted=convert_units_array(df_clipped['flux_foss'], "carbon_dioxide")

In [ ]:
# Cool but not interactive
sns.lineplot(
    y=da_converted.values,
    x=da_converted.coords['time.year']
)

In [ ]:
# Xarray doesn't cover all pandas functionalities, we need to convert it to a pandas dataframe
unit=da_converted.attrs["units"]
df_pandas = pd.DataFrame(da_converted.to_pandas(), columns=["flux_foss"]).reset_index()
df_pandas['year'] = df_pandas['time'].dt.year
df_pandas = df_pandas.groupby('year').agg(mean=('flux_foss', 'mean'), ci=('flux_foss', lambda d: sms.DescrStatsW(d).tconfint_mean()))
df_pandas[['lower', 'upper']] = pd.DataFrame(df_pandas['ci'].to_list(), index=df_pandas.index)

In [ ]:
# Plotly plot, it's interactive, some tweaking needed for the theme
times = df_pandas.index.tolist()
times_rev = times[::-1]

# Line 1
y1 = df_pandas['mean'].to_list()
y1_upper = df_pandas['upper'].to_list()
y1_lower = df_pandas['lower'].to_list()
y1_lower = y1_lower[::-1]

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=times+times_rev,
    y=y1_upper+y1_lower,
    fill='toself',
    fillcolor='rgba(0,100,200,0.2)',
    line_color='rgba(0,100,200,0.2)',
    showlegend= False
))
fig.add_trace(go.Scatter(
    x=times,
    y=y1,
    line_color='rgb(0,100,200)',
    name="fossile flux"
))
fig.update_traces(mode='lines')
fig.update_xaxes(title='years')
fig.update_yaxes(title=unit)
fig.update_layout(title= {"text":"Fossile fluxes in Italy", "x":0.45, "y": 0.85, "automargin":True, "yref":"container", "font":{"size":19}})
fig.show()

In [ ]:
# TODO:
## 1 - Shiftare lat e long se necessario -> EAC4 va da 0 a 360, ma Inversion è già tra -180+180
## 2 - Clip paese -> Capire se funziona, sembra funzionare
## 3 - Media annuale -> In realtà ci dovrebbe essere un modo per calcolare il CI su plotly, basta avere diversi valori per anno
##                    -> No, quella è seaborn, su plotly va fatto a mano usando ad esempio statsmodels
## 4 - Plot con CI al 95% e aggiugere la seconda linea

In [ ]:
manager2 = EAC4Instance(
    'total_column_nitrogen_dioxide',
    'netcdf',
    dates_range = '2018-01-01/2020-01-01',
    time_values = '00:00'
)
manager2.download()

In [ ]:
df2 = xr.open_dataset(manager2.file_full_path)
df2

In [ ]:
df2 = df2.rio.write_crs('EPSG:4326')

In [ ]:
#all_touched=True questo parametro include tutti i pixel toccati dal poligono definito, se False include solo i pixel il cui centro è incluso nel poligono
#approvato all_touched=True
df2_clipped = df2.rio.clip(sh[sh['ADMIN'] == 'Italy'].geometry.apply(mapping), sh.crs, drop=True, all_touched=True)

In [ ]:
df2_agg = df2_clipped.mean(dim=['latitude', 'longitude']).resample(time="1MS", restore_coord_dims=True).mean(dim='time')

In [ ]:
reference_value = df2_agg.mean(dim='time')

In [ ]:
reference_value

In [ ]:
df2_agg

In [ ]:
df2_agg_convert = convert_units_array(df2_agg['tcno2'], 'total_column_nitrogen_dioxide')

In [ ]:
reference_value = df2_agg_convert.mean().values

In [ ]:
df2_agg_anomalies = df2_agg_convert - reference_value
df2_agg_anomalies.attrs = df2_agg_convert.attrs

In [ ]:
df2_agg_convert

In [ ]:
fig = px.line(
    y=df2_agg_anomalies.values,
    x=df2_agg_anomalies.coords['time'],
    markers='o'
)
fig.update_xaxes(title="Month")
fig.update_yaxes(title=df2_agg_anomalies.attrs['units'])
fig.update_layout(
    title={
        "text": "Total columns N02 above Italy (anomalies)",
        "x": 0.45,
        "y": 0.95,
        "automargin": True,
        "yref": "container",
        "font": {
            "size":19
        }
    }
)

## Third plot

In [ ]:
fig = eac4_hovmoeller_levels_plot(
    'ozone',
    'go3',
    '2021-01-01/2021-04-01',
    '00:00',
    ['1','5', '10', '50', '100', '250', '500', '1000'],
    ['Italy'],
    "Ozone levels above Italy"
)
fig.show()

## Fourth plot
Hovmoeller O3

In [ ]:
o3_manager = EAC4Instance(
    'total_column_ozone',
    'netcdf',
    dates_range = '2020-01-01/2022-12-31',
    time_values = '00:00',
)
o3_manager.download()

In [ ]:
df_o3 = xr.open_dataset(o3_manager.file_full_path)
df_o3 = df_o3.rio.write_crs('EPSG:4326')
df_o3

In [ ]:
df_o3_agg = df_o3.resample(time="1MS", restore_coord_dims=True).mean(dim='time').mean(dim='longitude')
df_o3_convert = convert_units_array(df_o3_agg['gtco3'], 'total_column_ozone')

In [ ]:
df_o3_convert

In [ ]:
my_colorsc=[[0, 'green'], [0.5, 'red'], [1.0, 'rgb(0, 0, 255)']]

In [ ]:
fig = px.imshow(df_o3_convert.T, color_continuous_scale='Jet', origin='lower')
fig.show()

In [ ]:
color_scale = px.colors.sequential.Jet
num_colors = 16
if num_colors > len(color_scale):
    interpolated_colors = []
    for i in range(num_colors):
        index = int(i * (len(color_scale) - 1) / (num_colors - 1))
        interpolated_colors.append(color_scale[index])
    discrete_colors = interpolated_colors
else:
    discrete_colors = color_scale

# import matplotlib.colors as mcolors
# if num_colors > len(color_scale):
#     cmap = mcolors.LinearSegmentedColormap.from_list('custom_color_scale', color_scale)
#     interpolated_colors = [mcolors.rgb2hex(cmap(i)) for i in np.linspace(0, 1, num_colors)]
#     discrete_colors = interpolated_colors
# else:
#     discrete_colors = color_scale

# if num_colors > len(color_scale):
#     discrete_colors = []
#     for i in range(num_colors):
#         ratio = i / (num_colors - 1)
#         idx = int(ratio * (len(color_scale) - 1))
#         discrete_colors.append(color_scale[idx])
# else:
#     discrete_colors = color_scale[:num_colors]

# import matplotlib.colors as mcolors
# cmap = mcolors.LinearSegmentedColormap.from_list('custom_color_scale', color_scale)
# interpolated_colors = [mcolors.rgb2hex(cmap(i / (num_colors - 1))) for i in range(num_colors)]

""" import plotly.colors as pcolors
if num_colors > len(color_scale):
    interpolated_colors = pcolors.interpolate_colorscale(color_scale, num_colors)
    discrete_colors = [c for c, _ in interpolated_colors]
else:
    discrete_colors = color_scale[:num_colors] """

fig = px.imshow(df_o3_convert.T, color_continuous_scale=discrete_colors)
fig.show()

## Plot 3


In [ ]:
multilevel_manager = EAC4Instance(
    'ozone',
    'netcdf',
    pressure_level =['1','5', '10', '50', '100', '250', '500', '1000'],
    dates_range = '2020-01-01/2020-06-30',
    time_values = '00:00',
)
multilevel_manager.download()

In [ ]:
print(multilevel_manager.file_full_path)

In [ ]:
df_ml = xr.open_dataset(multilevel_manager.file_full_path)
df_ml = df_ml.rio.write_crs('EPSG:4326')
df_ml

In [ ]:
df_ml_shifted=ds_swaplon(df_ml)

In [ ]:
sh_down = ShapefilesDownloader(
    resolution='10m',
    instance='countries'
)
sh_down.download_shapefile()

In [ ]:
sh = gpd.read_file(sh_down.shapefile_full_path, crs='EPSG:4326')

In [ ]:

df_ml_clipped = df_ml_shifted.rio.clip(sh[sh['ADMIN'] == 'Italy'].geometry.apply(mapping), sh.crs, drop=True, all_touched=True)

In [ ]:
df_ml_agg = df_ml_shifted.resample(time="1W", restore_coord_dims=True).mean(dim='time').mean(dim='longitude').mean(dim='latitude').sortby("level")
df_ml_convert = convert_units_array(df_ml_agg['go3'], 'ozone')

In [ ]:
df_ml_convert

In [ ]:
fig = px.imshow(df_ml_convert.T, color_continuous_scale='RdBu_r', origin='lower')
fig.update_yaxes(type='log', autorange='reversed')
fig.update_layout(
    title={
        "text": "Italy Vertical hovmoeller plots of ozone",
        "x": 0.45,
        "y": 0.95,
        "automargin": True,
        "yref": "container",
        "font": {
            "size":19
        }
    }
)
fig.show()